### Query 3

    “Powerline Frequency” 60                <-- Acquisition Object Header
    “Race” Asian                            <-- Subject Level Form
    “Eye Dominance” Right                   <-- Subject Level Form
    “Institute” University of Birmingham    <-- Project Level Form
    “Grant ID” TWCF0389                     <-- Project Level Form
    
### Query 3 output parameters required

    project, subject, session



In [1]:
import pandas as pd
import setup
import nexussdk as nexus
from SPARQLWrapper import SPARQLWrapper, JSON, POST, GET, POSTDIRECTLY, CSV
from questionnaire_query_helper import generate_sparql_compliant_variable_name, generate_question_hash, query_constructor
config = setup.config
import NexusSparqlQuery as qns

# get script arguments
nexus_deployment = config['nexus']['deployment']
org = config['nexus']['org']
project = config['nexus']['project']
token = config['nexus']['token_file']
f = open(token, 'r')
token = f.read().rstrip()

sparqlview_endpoint = setup.sparqlview_endpoint
sparqlview_wrapper = setup.sparqlview_wrapper

nexus = setup.nexus

print("Connected")


Connected


In [2]:
prefix = """
prefix nxv: <https://bluebrain.github.io/nexus/vocabulary/>
PREFIX fhir: <http://hl7.org/fhir/> 
PREFIX kcnietl: <http://www.camh.ca/kcnietl/> 
PREFIX nidm: <http://purl.org/nidash/nidm#> 
PREFIX prov: <http://www.w3.org/ns/prov#> 
PREFIX sio: <http://semanticscience.org/resource/> 
PREFIX dct: <http://purl.org/dc/terms/> 
PREFIX bids: <https://wolfborg.github.io/nidm/bids.html#>
"""

select_list = [
    '?study_id',
    '?subject_id',
    '?session_id',
    '?acquisition_id',
    '?device_manufacturer',
    '?powerLineFrequency'
]

base_query = """
        ##### Get Study Information
        ?study_uri a fhir:ResearchStudy .
        ?study_uri fhir:ResearchStudy.identifier/fhir:Identifier.value/fhir:value ?study_id .
        
        #### Get Subject Information        
        ?subject_uri fhir:ResearchSubject.study/fhir:link ?study_uri .
        ?subject_uri fhir:ResearchSubject.identifier/fhir:Identifier.value/fhir:value ?subject_id .
        
        #### Get Session Information
        ?session a nidm:Session .
        ?session prov:wasAssociatedWith ?subject_uri .
        ?session dct:identifier ?session_id .
        
        #### Get Acquisition Information
        ?acquisition a nidm:Acquisition .
        ?acquisition dct:isPartOf ?session .
        ?acquisition dct:identifier ?acquisition_id .
        
        #### Get Acquisition Object Information
        ?acquisition_object a nidm:AcquisitionObject .
        ?acquisition_object prov:wasGeneratedBy ?acquisition .
        ?acquisition_object bids:powerLineFrequency ?powerLineFrequency .
        
"""

project_filter_list = ['Institute', 'Grant ID']
for question in project_filter_list:
    sparql_compliant_variable_name = generate_sparql_compliant_variable_name(question)
    select_list.append(sparql_compliant_variable_name)
    base_query = base_query + query_constructor('?study_uri', question, sparql_compliant_variable_name)
    
    
subject_question_list = ['Eye Dominance', 'Psychiatric History', 'Race']
for question in subject_question_list:
    sparql_compliant_variable_name = generate_sparql_compliant_variable_name(question)
    select_list.append(sparql_compliant_variable_name)
    base_query = base_query + query_constructor('?subject_uri', question, sparql_compliant_variable_name)


In [3]:
query = f"""
{prefix}

select distinct
    {' '.join(select_list)}
where
    {{
        {base_query}
    }}
"""

df = qns.sparql2dataframe(qns.query_sparql(query, sparqlview_wrapper))

df.loc[
    (df['powerLineFrequency'] == '50')
    & (df['Race'] == 'asian')
    & (df['Eye___Dominance'] == 'right')
    & (df['Institute'] == 'University of Birmingham' )
    & (df['Grant___ID'] == 'TWCF0389' )
    ].drop_duplicates()

,study_id,subject_id,session_id,acquisition_id,device_manufacturer,powerLineFrequency,Institute,Grant___ID,Eye___Dominance,Psychiatric___History,Race
17,nexus1,NA002,NA002_MEEG_1,RestingEO,None,50,University of Birmingham,TWCF0389,right,depression,asian
19,nexus1,NA002,NA002_MEEG_1,RestingEO,None,50,University of Birmingham,TWCF0389,right,None,asian
21,nexus1,NA001,NA001_MEEG_1,DurR1,None,50,University of Birmingham,TWCF0389,right,depression,asian
23,nexus1,NA001,NA001_MEEG_1,DurR1,None,50,University of Birmingham,TWCF0389,right,None,asian


“Powerline Frequency” 50                <-- Acquisition Object Header
“Race” Asian                            <-- Subject Level Form
“Eye Dominance” Right                   <-- Subject Level Form
“Institute” University of Birmingham    <-- Project Level Form
“Grant ID” TWCF0389                     <-- Project Level Form